# Import libraries

In [5]:
import numpy as np
import pandas as pd

# Import data

In [8]:
df = pd.read_csv("data/export/net_amount_discount_monthly.csv", header=0)
df.head()

,Purchase Date,Amount Type,Amount
0,2019-09-01,Net Amount,9.253245e+05
1,2019-09-01,Discount Amount INR,4.051689e+04
2,2019-10-01,Net Amount,2.698058e+06
3,2019-10-01,Discount Amount INR,1.376074e+05
4,2019-11-01,Net Amount,2.713950e+06


# Data Manipulation

 - Extend the data to create a complete timeline of the data
 - Interpolate missing values

In [9]:
# Convert Purchase Date to datetime
df["Purchase Date"] = pd.to_datetime(df["Purchase Date"])

# Get unique amount types
amount_types = df["Amount Type"].unique()

# Create an empty dataframe to store the results
result_df = pd.DataFrame()

# Process each amount type
for amount_type in amount_types:
    # Filter data for current amount type
    type_data = df[df["Amount Type"] == amount_type].copy()
    # Get min and max dates for this amount type
    min_date = type_data["Purchase Date"].min()
    max_date = type_data["Purchase Date"].max()
    # Create date range specific to this amount type
    date_range = pd.date_range(start=min_date, end=max_date, freq="MS")
    # Create complete timeline for this amount type
    complete_df = pd.DataFrame({"Purchase Date": date_range})
    complete_df = complete_df.merge(type_data, on="Purchase Date", how="left")
    # Fill missing Amount Type
    complete_df["Amount Type"] = amount_type
    # Fill missing values with NaN
    complete_df["Amount"] = complete_df["Amount"].fillna(np.nan)
    # Interpolate missing values
    complete_df["Amount"] = complete_df["Amount"].interpolate(
        method="linear", limit_direction="both"
    )
    # Append to the result dataframe
    result_df = pd.concat([result_df, complete_df])

# Sort the final dataframe
result_df = result_df.sort_values(["Purchase Date", "Amount Type"]).reset_index(
    drop=True
)